In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [3]:
tf.enable_eager_execution()

In [4]:
! pip install -q requests
! git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3050, done.
remote: Counting objects: 100% (3050/3050), done.
remote: Compressing objects: 100% (2565/2565), done.
remote: Total 3050 (delta 541), reused 2054 (delta 408), pack-reused 0
Receiving objects: 100% (3050/3050), 370.25 MiB | 2.44 MiB/s, done.
Resolving deltas: 100% (541/541), done.
Checking out files: 100% (2883/2883), done.


In [6]:
model_path=os.path.join(os.getcwd(),'models')

In [7]:
model_path

'/Users/akshaybrijpuria/Documents/PyScript/models'

In [9]:
sys.path.append(model_path)

In [10]:
from official.wide_deep import census_dataset
from official.wide_deep import census_main

In [11]:
census_dataset.download('/tmp/census_dataset')

In [12]:
!ls /tmp/census_dataset

adult.data adult.test


In [13]:
train_file = "/tmp/census_dataset/adult.data"
test_file = "/tmp/census_dataset/adult.test"

In [16]:
import pandas as pd
train = pd.read_csv(train_file,names=census_dataset._CSV_COLUMNS)
test = pd.read_csv(test_file,names=census_dataset._CSV_COLUMNS)

train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [17]:
def easy_input(df,label_key,num_epochs, shuffle, batch_size):
    label = df[label_key]
    ds = tf.data.Dataset.from_tensor_slices((dict(df),label))
    if shuffle:
        ds = ds.shuffle(10000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
    

In [18]:
ds = easy_input(train,label_key='income_bracket',num_epochs=5,shuffle=True,batch_size=10)

In [30]:
for feature,label in ds.take(1):
    print(list(feature))

['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income_bracket']


In [90]:
ds = census_dataset.input_fn(train_file,num_epochs=2,shuffle=True,batch_size=16)
for ft,lt in ds.take(1):
    print(ft.keys())


INFO:tensorflow:Parsing /tmp/census_dataset/adult.data


I0428 02:01:10.292570 4406461888 census_dataset.py:167] Parsing /tmp/census_dataset/adult.data


dict_keys(['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country'])


In [22]:
import inspect
print(inspect.getsource(census_dataset.input_fn))

def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have run census_dataset.py and '
      'set the --data_dir argument to the correct path.' % data_file)

  def parse_csv(value):
    tf.logging.info('Parsing {}'.format(data_file))
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('income_bracket')
    classes = tf.equal(labels, '>50K')  # binary classification
    return features, classes

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = 

In [23]:
import functools
train_fn = functools.partial(census_dataset.input_fn,train_file,num_epochs=2,shuffle=True,batch_size=64)
test_fn = functools.partial(census_dataset.input_fn,test_file,num_epochs=1,shuffle=False,batch_size=64)

In [24]:
import tensorflow.feature_column as fc
age = fc.numeric_column('age')

In [33]:
type(feature)

dict

In [35]:
fc.input_layer(feature,[age]).numpy()

array([[49.],
       [48.],
       [32.],
       [37.],
       [41.],
       [33.],
       [20.],
       [52.],
       [19.],
       [45.]], dtype=float32)

In [37]:
classifier = tf.estimator.LinearClassifier(feature_columns=[age])
classifier.train(train_fn)
result = classifier.evaluate(test_fn)
print(result)

INFO:tensorflow:Using default config.


I0428 00:56:10.970901 4406461888 estimator.py:1739] Using default config.


W0428 00:56:10.974803 4406461888 estimator.py:1760] Using temporary folder as model directory: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmp6un3wljn


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmp6un3wljn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ca237f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0428 00:56:10.977743 4406461888 estimator.py:201] Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmp6un3wljn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ca237f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving dict for global step 1018: accuracy = 0.76211536, accuracy_baseline = 0.76377374, auc = 0.6782223, auc_precision_recall = 0.31135228, average_loss = 0.52340305, global_step = 1018, label/mean = 0.23622628, loss = 33.417747, precision = 0.2244898, prediction/mean = 0.23650634, recall = 0.0028601144


I0428 00:56:19.667680 4406461888 estimator.py:1979] Saving dict for global step 1018: accuracy = 0.76211536, accuracy_baseline = 0.76377374, auc = 0.6782223, auc_precision_recall = 0.31135228, average_loss = 0.52340305, global_step = 1018, label/mean = 0.23622628, loss = 33.417747, precision = 0.2244898, prediction/mean = 0.23650634, recall = 0.0028601144


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmp6un3wljn/model.ckpt-1018


I0428 00:56:19.860394 4406461888 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmp6un3wljn/model.ckpt-1018


{'accuracy': 0.76211536, 'accuracy_baseline': 0.76377374, 'auc': 0.6782223, 'auc_precision_recall': 0.31135228, 'average_loss': 0.52340305, 'label/mean': 0.23622628, 'loss': 33.417747, 'precision': 0.2244898, 'prediction/mean': 0.23650634, 'recall': 0.0028601144, 'global_step': 1018}


In [39]:
educational_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_week = tf.feature_column.numeric_column('hours_per_week')
fc.input_layer(feature,[educational_num,capital_gain,capital_loss,hours_week])

<tf.Tensor: id=2183, shape=(10, 4), dtype=float32, numpy=
array([[   0.,    0.,   11.,   60.],
       [   0., 1672.,    4.,   55.],
       [   0.,    0.,    9.,   40.],
       [   0.,    0.,   10.,   40.],
       [   0.,    0.,    9.,   40.],
       [   0.,    0.,   13.,   40.],
       [   0.,    0.,    9.,   40.],
       [   0.,    0.,    9.,   40.],
       [   0.,    0.,   10.,   25.],
       [   0.,    0.,   13.,   46.]], dtype=float32)>

In [46]:
classifier = tf.estimator.LinearClassifier(feature_columns=[educational_num,capital_gain,capital_loss,hours_week])
classifier.train(train_fn)
res = classifier.evaluate(test_fn)
print(res)

INFO:tensorflow:Using default config.


I0428 01:06:26.243548 4406461888 estimator.py:1739] Using default config.


W0428 01:06:26.248347 4406461888 estimator.py:1760] Using temporary folder as model directory: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpecu_820n


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpecu_820n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f45aa58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0428 01:06:26.251269 4406461888 estimator.py:201] Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpecu_820n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f45aa58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving dict for global step 1018: accuracy = 0.7625453, accuracy_baseline = 0.76377374, auc = 0.58683616, auc_precision_recall = 0.38979068, average_loss = 0.89920026, global_step = 1018, label/mean = 0.23622628, loss = 57.411293, precision = 0.48687664, prediction/mean = 0.21404573, recall = 0.09646386


I0428 01:06:36.992847 4406461888 estimator.py:1979] Saving dict for global step 1018: accuracy = 0.7625453, accuracy_baseline = 0.76377374, auc = 0.58683616, auc_precision_recall = 0.38979068, average_loss = 0.89920026, global_step = 1018, label/mean = 0.23622628, loss = 57.411293, precision = 0.48687664, prediction/mean = 0.21404573, recall = 0.09646386


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpecu_820n/model.ckpt-1018


I0428 01:06:37.258570 4406461888 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpecu_820n/model.ckpt-1018


{'accuracy': 0.7625453, 'accuracy_baseline': 0.76377374, 'auc': 0.58683616, 'auc_precision_recall': 0.38979068, 'average_loss': 0.89920026, 'label/mean': 0.23622628, 'loss': 57.411293, 'precision': 0.48687664, 'prediction/mean': 0.21404573, 'recall': 0.09646386, 'global_step': 1018}


In [49]:
classifier = tf.estimator.LinearClassifier(feature_columns=[educational_num,capital_gain,capital_loss,hours_week])
ft = functools.partial(census_dataset.input_fn,train_file,num_epochs=2,shuffle=True,batch_size=3)


INFO:tensorflow:Using default config.


I0428 01:10:01.555724 4406461888 estimator.py:1739] Using default config.


W0428 01:10:01.560922 4406461888 estimator.py:1760] Using temporary folder as model directory: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpjjpuh7a_


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpjjpuh7a_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f163898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0428 01:10:01.569688 4406461888 estimator.py:201] Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpjjpuh7a_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f163898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
classifier.train(ft)

INFO:tensorflow:Parsing /tmp/census_dataset/adult.data


I0428 01:10:31.600404 4406461888 census_dataset.py:167] Parsing /tmp/census_dataset/adult.data


INFO:tensorflow:Calling model_fn.


I0428 01:10:31.659542 4406461888 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

I0428 01:11:07.369274 4406461888 basic_session_run_hooks.py:247] loss = 0.3683181, step = 21701 (0.149 sec)


INFO:tensorflow:Saving checkpoints for 21708 into /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpjjpuh7a_/model.ckpt.


I0428 01:11:07.388922 4406461888 basic_session_run_hooks.py:594] Saving checkpoints for 21708 into /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpjjpuh7a_/model.ckpt.


INFO:tensorflow:Loss for final step: 0.1487645.


I0428 01:11:07.489332 4406461888 estimator.py:359] Loss for final step: 0.1487645.


In [53]:
relationship = fc.categorical_column_with_vocabulary_list('relationship',vocabulary_list=['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'])

In [60]:
fc.input_layer(feature,[age,fc.indicator_column(relationship)])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:16:46.392384 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:205: IndicatorColumn._get_dense_tensor (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:16:46.400602 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:2121: IndicatorColumn._transform_feature (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


...

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:16:46.432919 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4266: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:16:46.439973 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4321: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


<tf.Tensor: id=185182, shape=(10, 7), dtype=float32, numpy=
array([[49.,  1.,  0.,  0.,  0.,  0.,  0.],
       [48.,  1.,  0.,  0.,  0.,  0.,  0.],
       [32.,  1.,  0.,  0.,  0.,  0.,  0.],
       [37.,  0.,  0.,  0.,  0.,  1.,  0.],
       [41.,  1.,  0.,  0.,  0.,  0.,  0.],
       [33.,  1.,  0.,  0.,  0.,  0.,  0.],
       [20.,  0.,  0.,  0.,  0.,  0.,  1.],
       [52.,  1.,  0.,  0.,  0.,  0.,  0.],
       [19.,  0.,  0.,  0.,  1.,  0.,  0.],
       [45.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)>

In [61]:
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)

In [66]:
fc.input_layer(feature,[fc.indicator_column(occupation)])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:23:17.733595 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4295: HashedCategoricalColumn._get_sparse_tensors (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:23:17.736794 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:2121: HashedCategoricalColumn._transform_feature (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:23:17.742465 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:4321: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


<tf.Tensor: id=185209, shape=(10, 1000), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [67]:
education = tf.feature_column.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

In [69]:
lin_classifier = tf.estimator.LinearClassifier([age,relationship,occupation,education,marital_status,workclass])
lin_classifier.train(ft)
lin_classifier.evaluate(test_fn)

INFO:tensorflow:Using default config.


I0428 01:27:17.896862 4406461888 estimator.py:1739] Using default config.


W0428 01:27:17.904506 4406461888 estimator.py:1760] Using temporary folder as model directory: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpn4639d2w


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpn4639d2w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x132bc37f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0428 01:27:17.908638 4406461888 estimator.py:201] Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpn4639d2w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x132bc37f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving dict for global step 21708: accuracy = 0.8317671, accuracy_baseline = 0.76377374, auc = 0.8803973, auc_precision_recall = 0.6809083, average_loss = 0.3563082, global_step = 21708, label/mean = 0.23622628, loss = 22.74923, precision = 0.6953759, prediction/mean = 0.2280061, recall = 0.5122205


I0428 01:28:13.069788 4406461888 estimator.py:1979] Saving dict for global step 21708: accuracy = 0.8317671, accuracy_baseline = 0.76377374, auc = 0.8803973, auc_precision_recall = 0.6809083, average_loss = 0.3563082, global_step = 21708, label/mean = 0.23622628, loss = 22.74923, precision = 0.6953759, prediction/mean = 0.2280061, recall = 0.5122205


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21708: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpn4639d2w/model.ckpt-21708


I0428 01:28:13.354778 4406461888 estimator.py:2039] Saving 'checkpoint_path' summary for global step 21708: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpn4639d2w/model.ckpt-21708


{'accuracy': 0.8317671,
 'accuracy_baseline': 0.76377374,
 'auc': 0.8803973,
 'auc_precision_recall': 0.6809083,
 'average_loss': 0.3563082,
 'label/mean': 0.23622628,
 'loss': 22.74923,
 'precision': 0.6953759,
 'prediction/mean': 0.2280061,
 'recall': 0.5122205,
 'global_step': 21708}

In [70]:
age_bucket = tf.feature_column.bucketized_column(age,boundaries=[18,24,30,35,40,45,50,55,60,65])
fc.input_layer(feature,[age,age_bucket])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:35:44.756433 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:205: BucketizedColumn._get_dense_tensor (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:35:44.759781 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:2121: BucketizedColumn._transform_feature (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


W0428 01:35:44.764723 4406461888 deprecation.py:323] From /anaconda3/envs/tflow/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column.py:206: BucketizedColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed after 2018-11-30.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


<tf.Tensor: id=189019, shape=(10, 12), dtype=float32, numpy=
array([[49.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [48.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [32.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [37.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [41.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [33.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [20.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [52.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [19.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [45.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]],
      dtype=float32)>

In [71]:
education_x_occup = tf.feature_column.crossed_column(['occupation','education'],hash_bucket_size=1000)

In [76]:
fc.input_layer(feature,[fc.indicator_column(education),fc.indicator_column(education_x_occup)])

<tf.Tensor: id=189207, shape=(10, 1016), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [77]:
age_x_education_occup = tf.feature_column.crossed_column(['age','education','occupation'],hash_bucket_size=1000)

In [79]:
classifier = tf.estimator.LinearClassifier(feature_columns=[age,education,marital_status,occupation,relationship,workclass,education_x_occup,age_x_education_occup],
                                           optimizer=tf.train.FtrlOptimizer(learning_rate=0.1))

INFO:tensorflow:Using default config.


I0428 01:53:05.843667 4406461888 estimator.py:1739] Using default config.


W0428 01:53:05.847847 4406461888 estimator.py:1760] Using temporary folder as model directory: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f872ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0428 01:53:05.850723 4406461888 estimator.py:201] Using config: {'_model_dir': '/var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f872ac8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [80]:
classifier.train(train_fn)
classifier.evaluate(test_fn)

INFO:tensorflow:Parsing /tmp/census_dataset/adult.data


I0428 01:53:33.381050 4406461888 census_dataset.py:167] Parsing /tmp/census_dataset/adult.data


INFO:tensorflow:Calling model_fn.


I0428 01:53:33.432463 4406461888 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 1018: accuracy = 0.8296788, accuracy_baseline = 0.76377374, auc = 0.87976885, auc_precision_recall = 0.6831044, average_loss = 0.35869595, global_step = 1018, label/mean = 0.23622628, loss = 22.901682, precision = 0.66292137, prediction/mean = 0.2570632, recall = 0.5676027


I0428 01:53:51.412678 4406461888 estimator.py:1979] Saving dict for global step 1018: accuracy = 0.8296788, accuracy_baseline = 0.76377374, auc = 0.87976885, auc_precision_recall = 0.6831044, average_loss = 0.35869595, global_step = 1018, label/mean = 0.23622628, loss = 22.901682, precision = 0.66292137, prediction/mean = 0.2570632, recall = 0.5676027


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji/model.ckpt-1018


I0428 01:53:51.774458 4406461888 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1018: /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji/model.ckpt-1018


{'accuracy': 0.8296788,
 'accuracy_baseline': 0.76377374,
 'auc': 0.87976885,
 'auc_precision_recall': 0.6831044,
 'average_loss': 0.35869595,
 'label/mean': 0.23622628,
 'loss': 22.901682,
 'precision': 0.66292137,
 'prediction/mean': 0.2570632,
 'recall': 0.5676027,
 'global_step': 1018}

In [142]:
test_df = test[:20].copy()

In [143]:
pred_iter = classifier.predict(lambda:easy_input(test_df,label_key='income_bracket',num_epochs=1,shuffle=True,batch_size=10))

In [144]:
classes=np.array(['<=50K','>50K'])

In [145]:
pred_class_id=[]
for preds in pred_iter:
    pred_class_id.append(preds['class_ids'])

    

INFO:tensorflow:Calling model_fn.


I0428 02:37:02.985882 4406461888 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Done calling model_fn.


I0428 02:37:03.778947 4406461888 estimator.py:1113] Done calling model_fn.


INFO:tensorflow:Graph was finalized.


...

I0428 02:37:03.955410 4406461888 saver.py:1270] Restoring parameters from /var/folders/6v/4p9b28qd1s9ckw__97m6_v3m0000gn/T/tmpd4d179ji/model.ckpt-1018


INFO:tensorflow:Running local_init_op.


I0428 02:37:04.051249 4406461888 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0428 02:37:04.082134 4406461888 session_manager.py:493] Done running local_init_op.


In [146]:
test_df['Predicted_class'] = classes[np.array(pred_class_id)]

In [150]:
test_df['Correct'] = test_df['Predicted_class'] == test_df['income_bracket']

In [151]:
test_df[['income_bracket','Predicted_class','Correct']]

,income_bracket,Predicted_class,Correct
0,<=50K,<=50K,True
1,<=50K,<=50K,True
2,>50K,>50K,True
3,>50K,<=50K,False
4,<=50K,<=50K,True
5,<=50K,<=50K,True
6,<=50K,<=50K,True
7,>50K,<=50K,False
8,<=50K,<=50K,True
9,<=50K,<=50K,True
